In [1]:
%load_ext autoreload
%autoreload 2

## **[edge attr]** *euclidean and normalized direction* 
## **[edge net]** *2 layers*
## **[block dim]** *8,8,8,8*
## **[num blocks]** *9*
## **[init pos]** *scaled random*
## **[loss]** *StressLoss:1, RescaledL2AngularLoss:0.01*

In [2]:
from lib.imports import *
from lib.functions import *
from lib.utils import *
from lib.modules import *
from ipynb.fs.defs.losses import *

In [3]:
torch.__version__

'1.5.0'

In [4]:
torch.cuda.device_count()

2

# Settings

In [5]:
cuda_idx = 0
cfg_file = 'gnn_stress-Copy74.json'
loss_fns = {
    StressLoss(): 1, RescaledL2AngularLoss(): 0.01
}

In [6]:
config = Config(cfg_file)
config[...]

{'name': 'scaled_l2:0.01',
 'batchsize': 128,
 'epoch': {'start': 275, 'end': None},
 'lr': {'initial': 0.01,
  'decay_rate': 0.99,
  'decay_step': 1,
  'override': {'start_epoch': None,
   'target_lr': None,
   'decay_rate': None,
   'decay_step': None}},
 'log_period': 1,
 'test': {'name': 'scaled_l2:0.01', 'epoch': 567}}

In [7]:
print(f"cd {os.getcwd()}")
print(f"tail -f ./logs/{config.name}.log")

cd /home/jupyter/graph-drawing-stress
tail -f ./logs/scaled_l2:0.01.log


In [8]:
device = f'cuda:{cuda_idx}' if cuda_idx is not None and torch.cuda.is_available() else 'cpu'
nvidia_smi.nvmlInit()
cuda = nvidia_smi.nvmlDeviceGetHandleByIndex(cuda_idx) if cuda_idx is not None else None
np.set_printoptions(precision=2)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [9]:
def get_lr(epoch):
    current_epoch = config.epoch.start + epoch
    if config.lr.override.start_epoch is not None and current_epoch >= config.lr.override.start_epoch:
        num_step = (current_epoch - config.lr.override.start_epoch) // config.lr.override.decay_step
        return config.lr.override.target_lr * config.lr.override.decay_rate ** num_step
    num_step = current_epoch // config.lr.decay_step
    return config.lr.initial * config.lr.decay_rate ** num_step

In [10]:
G_list, data_list = load_processed_data(G_list_file='G_list_1.pickle', 
                                        data_list_file='data_list_1.pickle',
                                        index_file='data_index_1.txt')
train_loader = DataLoader(data_list[:10000], batch_size=config.batchsize, shuffle=True)
val_loader = DataLoader(data_list[11000:], batch_size=config.batchsize, shuffle=False)

In [11]:
criterion = CompositeLoss(loss_fns.keys(), weights=[loss_fns[fn] for fn in loss_fns])

# Training

In [ ]:
if not os.path.isdir(f"../ckpt_{config.name}"):
    os.mkdir(f"../ckpt_{config.name}")
model = Model().to(device)
if config.epoch.start > 0:
    state_dict = torch.load(f"../ckpt_{config.name}/epoch_{config.epoch.start}.pt")
    model.load_state_dict(state_dict)
optimizer = torch.optim.AdamW(model.parameters(), lr=1)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=get_lr)
loss_ep = []
print("=" * 50, file=open(f"{config.name}.log", "a"))
epoch = config.epoch.start + 1
with tqdm(total=len(train_loader), smoothing=0) as progress:
    while True:
        progress.reset()
        progress.set_description(desc=f"[epoch {epoch}/{config.epoch.end}]")
        train_loss, train_loss_comp = train(model, criterion, optimizer, train_loader, device, progress, cuda, output_components=True)
        loss_ep.append(train_loss)
        lr = scheduler.get_lr()
        scheduler.step()
        if epoch % config.log_period == 0:
            torch.save(model.state_dict(), f"../ckpt_{config.name}/epoch_{epoch}.pt")
            val_losses, val_losses_comp = test(model, criterion, val_loader, device, output_components=True)
            print(f'{epoch}, train: {train_loss, train_loss_comp.tolist()}, val: {val_losses, val_losses_comp.tolist()}, lr: {lr}', 
                  file=open(f"{config.name}.log", "a"))
        if epoch == config.epoch.end:
            break
        epoch += 1

# Performance Testing

In [12]:
state_dict = torch.load(f'./checkpoints/{config.name}/epoch_{config.test.epoch}.pt', map_location=torch.device(device))
model = Model().to(device)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [13]:
test_metrics = test(model=model, 
                    criteria_list=list(loss_fns.keys()), 
                    dataset=data_list, 
                    idx_range=range(10000, 11000), 
                    callback=None)

In [14]:
if not os.path.isdir(f"./metrics"):
    os.mkdir("./metrics")
pickle.dump(test_metrics, open(f"./metrics/{config.name}_test.pickle", "wb"))

/opt/conda/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


##### criterion = CompositeLoss([fn() for fn in loss_fns], weights=[loss_fns[fn] for fn in loss_fns])
folder_name = f'{config.test.name}_test'
test_loss = []
test_loss_ratio=[]
reso_scores = []
ring_loss = []
reso_loss=[]
minOfMin = []
stress_criterion = StressLoss()
for test_idx in tqdm(range(10000,11000)):
    G = G_list[test_idx]
    data = data_list[test_idx]
    node_pos, loss,component = evaluate(model, data, criterion, device,output_components=True)
    node_pos_tensor = torch.tensor(node_pos).to(device)
#     node_pos_tensor = rescale_with_minimized_stress(node_pos_tensor, data)
#     stress_loss = stress_criterion(node_pos_tensor, data)
    theta,degree,u = get_theta_angles_and_node_degrees(node_pos_tensor, data,return_u=True)
    gt_loss = ground_truth.loc[test_idx][0]
    loss_ratio = (component[0] - gt_loss) / gt_loss
    minDegree = (theta.min().item()/2*np.pi)*360
    minOfMin.append(minDegree)
    score = resolution_score(theta,degree,u)
    test_loss.append(component[0])
    reso_scores.append(score) 
    reso_loss.append(component[1])
    test_loss_ratio.append(loss_ratio)
    
#     graph_vis(G, node_pos, f'{folder_name}/{test_idx}_{component[0]:.2f}_{score:.2f}.png')  
#     node_pos = nx.nx_agraph.graphviz_layout(G_vis, prog='neato')
#     plt.figure()
#     nx.draw(G_vis, node_pos)
#     plt.savefig(f'{folder_name}/{test_idx}.png')


In [17]:
print("stress", np.nanmean(test_loss), np.nanstd(test_loss))
print("reso loss", np.nanmean(reso_loss), np.nanstd(reso_loss))
print("score",np.nanmean(reso_scores), np.nanstd(reso_scores))
print("min degree", np.nanmean(minOfMin), np.nanstd(minOfMin))
print("loss ratio", torch.tensor(test_loss_ratio).mean(), torch.tensor(test_loss_ratio).std())

stress 128.13498 132.22224
reso loss 3950.8694 3465.1848
score 0.6184993895292282 0.08608516239465049
min degree 65.92132150817764 122.48960832630101
loss ratio tensor(0.1121) tensor(0.4225)


In [19]:
print("stress", np.nanmean(test_loss), np.nanstd(test_loss))
print("reso loss", np.nanmean(reso_loss), np.nanstd(reso_loss))
print("score",np.nanmean(reso_scores), np.nanstd(reso_scores))
print("min degree", np.nanmean(minOfMin), np.nanstd(minOfMin))
print("loss ratio", torch.tensor(test_loss_ratio).mean(), torch.tensor(test_loss_ratio).std())

stress 128.44609 132.32016
reso loss 3948.49 3461.4526
score 0.6185177138149738 0.08393357852062479
min degree 64.49263581112193 112.43025634932545
loss ratio tensor(0.1281, dtype=torch.float64) tensor(0.4294, dtype=torch.float64)


In [20]:
criterion = CompositeLoss([fn() for fn in loss_fns], weights=[loss_fns[fn] for fn in loss_fns])
folder_name = f'{config.test.name}_test'
test_loss = []
test_loss_ratio=[]
reso_scores = []
ring_loss = []
reso_loss=[]
minOfMin = []
for test_idx in tqdm(range(10000)):
    G = G_list[test_idx]
    data = data_list[test_idx]
    node_pos, loss,component = evaluate(model, data, criterion, device,output_components=True)
    node_pos_tensor = torch.tensor(node_pos).to(device)
#     node_pos_tensor = rescale_with_minimized_stress(node_pos_tensor, data)
    loss = criterion(node_pos_tensor, data)
    theta,degree,u = get_theta_angles_and_node_degrees(node_pos_tensor, data,return_u=True)
    gt_loss = ground_truth.loc[test_idx][0]
    loss_ratio = (component[0] - gt_loss) / gt_loss
    minDegree = (theta.min().item()/2*np.pi)*360
    minOfMin.append(minDegree)
    score = resolution_score(theta,degree,u)
    test_loss.append(component[0])
    reso_scores.append(score) 
    reso_loss.append(component[1])
    test_loss_ratio.append(loss_ratio)
    
#     graph_vis(G, node_pos, f'{folder_name}/{test_idx}_{component[0]:.2f}_{score:.2f}.png')  
#     node_pos = nx.nx_agraph.graphviz_layout(G_vis, prog='neato')
#     plt.figure()
#     nx.draw(G_vis, node_pos)
#     plt.savefig(f'{folder_name}/{test_idx}.png')


In [22]:
print("stress", np.nanmean(test_loss), np.nanstd(test_loss))
print("reso loss", np.nanmean(reso_loss), np.nanstd(reso_loss))
print("score",np.nanmean(reso_scores), np.nanstd(reso_scores))
print("min degree", np.nanmean(minOfMin), np.nanstd(minOfMin))
print("loss ratio", torch.tensor(test_loss_ratio).mean(), torch.tensor(test_loss_ratio).std())

stress 131.3693 134.98665
reso loss 4042.2537 3531.255
score 0.6248861903443933 0.08340612101093055
min degree 72.32880340359506 137.6819985040915
loss ratio tensor(0.2605, dtype=torch.float64) tensor(8.0769, dtype=torch.float64)


In [ ]:
model = Model().to(device)
state_dict = torch.load(f"../ckpt_{config.name}/epoch_{config.test.epoch}.pt",map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
test_idx = 10226
G = G_list[test_idx]
data = data_list[test_idx]
criterion = CompositeLoss([StressLoss(),AngularResolutionMAELossRescale()])
node_pos, loss,component = evaluate(model, data, criterion, device,output_components=True)
graph_vis(G, node_pos,node_size=600, with_labels=True, font_color="white", font_weight="bold", font_size=14) 


In [ ]:
losses = []
loss_ratios = []
for test_idx in tqdm(range(10000, 11000)):
    G_vis = G_list[test_idx]
    node_pos, loss = evaluate(model, data_list[test_idx], criterion, device)
    gt_loss = ground_truth.loc[test_idx][0]
    loss_ratio = (loss - gt_loss) / gt_loss
    losses += [loss]
    loss_ratios += [loss_ratio]

In [ ]:
np.mean(losses), np.std(losses)

In [ ]:
np.mean(loss_ratios), np.std(loss_ratios)

In [ ]:
truth_loss = 0
pred_loss = 0
for idx in tqdm(range(10000, 11000)):
    pred, loss = evaluate(model, data_list[idx], criterion, device)
    pos_map = nx.nx_agraph.graphviz_layout(G_list[idx], prog='neato')

    pred_mean, pred_std = pred.mean(axis=0), pred.std()
    truth = np.array(list(pos_map.values()))
    truth_mean, truth_std = truth.mean(axis=0), truth.std()
    norm_truth = (truth - truth_mean) / truth_std
    scaled_truth = norm_truth * pred_std + pred_mean

    truth_loss += criterion(torch.tensor(scaled_truth), data_list[idx])
    pred_loss += criterion(torch.tensor(pred), data_list[idx])
    
truth_loss / 1000, pred_loss / 1000

In [ ]:
type(data_list[9999].x)

In [ ]:
iterations = 5
losses = []
folder_name = f'{config.test.name}_iterative_test'
for test_idx in tqdm(range(10000, len(G_list))):
    G_vis = G_list[test_idx]
    node_pos = nx.nx_agraph.graphviz_layout(G_vis, prog='neato')
    plt.figure()
    nx.draw(G_vis, node_pos)
    plt.savefig(f'{folder_name}/{test_idx}.png')
    for i in range(iterations):
        node_pos, loss = evaluate(model, data_list[test_idx], criterion, device) 
        data_list[test_idx].x = torch.tensor(node_pos,dtype=torch.float)
    losses += [loss]
    graph_vis(G_vis, node_pos, f'{folder_name}/{config.test.out_prefix}_iter_model_{test_idx}_{loss}.png')

In [ ]:
plt.xlabel("epochs")
plt.ylabel("loss")
plt.plot(loss_ep[:1000])
plt.show()

In [ ]:
class EnergyLossScaled(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, p, data, scale):
        edge_attr = data.edge_attr
        # convert per-node positions to per-edge positions
        start, end, n_nodes = node2edge(p, data)
        
        start *= scale
        end *= scale
        
        start_x = start[:, 0]
        start_y = start[:, 1]
        end_x = end[:, 0]
        end_y = end[:, 1]
        
        l = edge_attr[:, 0]
        k = edge_attr[:, 1]
        
        term1 = (start_x - end_x) ** 2
        term2 = (start_y - end_y) ** 2
        term3 = l ** 2
        term4 = 2 * l * (term1 + term2).sqrt()
        energy = k / 2 * (term1 + term2 + term3 - term4)
        return energy.sum()

In [ ]:
criterion_scaled = EnergyLossScaled()
criterion = EnergyLossVectorized()

In [ ]:
truth_loss = 0
pred_loss = 0
for idx in tqdm(range(10000, 11000)):
    pred, loss = evaluate(model, data_list[idx], criterion, device)
    pos_map = nx.nx_agraph.graphviz_layout(G_list[idx], prog='neato')

    pred_mean, pred_std = pred.mean(axis=0), pred.std()
    truth = np.array(list(pos_map.values()))
    truth_mean, truth_std = truth.mean(axis=0), truth.std()
    norm_truth = (truth - truth_mean) / truth_std
    scaled_truth = norm_truth * pred_std + pred_mean

    truth_loss += criterion(torch.tensor(scaled_truth, device=device), data_list[idx])
    pred_loss += criterion(torch.tensor(pred, device=device), data_list[idx])
    
truth_loss / 1000, pred_loss / 1000

In [ ]:
for idx in [10898, 10904]:#tqdm(range(10000, 11000)):
    data, G = data_list[idx], G_list[idx]
    edge_attr = data.edge_attr
    pred, loss = evaluate(model, data, criterion, device)
    pos_map = nx.nx_agraph.graphviz_layout(G, prog='neato')
    truth = np.array(list(pos_map.values()))

    start, end, n_nodes = node2edge(torch.tensor(truth, device=device), data)
    w = edge_attr[:, 1]
    d = edge_attr[:, 0]

    u2 = ((start - end) ** 2).sum(dim=1)

    s = (w * d * u2.sqrt()).sum() / (w * u2).sum()

    loss_gt = criterion_scaled(torch.tensor(truth, device=device), data, s)

    print(loss, loss_gt)

In [ ]:
def evaluate(model, data, criterion, device, idx):
    model.eval()
    with torch.no_grad():
        data = data.to(device)
        pred = model(data).detach()
        loss = criterion(pred,data).cpu().numpy()
        loss = round(float(loss),2)
    return pred.cpu().numpy(), loss

def graph_vis(G, node_pos):
    i = 0
    for n, p in node_pos:
        node = 'n' +str(i)
        G.nodes[node]['pos'] = (n,p)
        i += 1
    pos = nx.get_node_attributes(G,'pos')
    plt.figure()
    nx.draw(G, pos)
    
for test_idx in tqdm(list(range(10000, len(data_list)))):
    G_vis = G_list[test_idx]
    node_pos,loss = evaluate(model, data_list[test_idx],criterion,device, test_idx)
    if loss > 10000:
        print(test_idx, loss, data_list[test_idx].num_nodes)
        graph_vis(G_vis, node_pos) 
        node_pos = nx.nx_agraph.graphviz_layout(G_vis, prog='neato')
        plt.figure()
        nx.draw(G_vis, node_pos)

# Eval with Projection

In [ ]:
idx = 10117
pred, loss, sloss, aloss = evaluate(model, data_list[idx], criterion, device, output_components=True, reduction=umap_project)
graph_vis(G_list[idx], pred, f'composite_umap_{idx}_loss_{sloss}_{aloss}.png')
print(sloss, aloss)

# Iterative Evaluation

In [ ]:
iterations = 100
losseses = []
folder_name = f'{config.test.name}_iterative_test'
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
for test_idx in tqdm(range(10000, 10500)):
    G_vis = G_list[test_idx]
#     node_pos = nx.nx_agraph.graphviz_layout(G_vis, prog='neato')
#     plt.figure()
#     nx.draw(G_vis, node_pos)
#     plt.savefig(f'{folder_name}/{test_idx}.png')
    losses = []
    for i in range(iterations):
        node_pos, loss, stress, angle = evaluate(model, data_list[test_idx], criterion, device, output_components=True, with_initial_pos=True) 
        data_list[test_idx].x = torch.tensor(node_pos,dtype=torch.float)
        losses += [loss]
    losseses += [losses]
    graph_vis(G_vis, node_pos, f'{folder_name}/{config.test.out_prefix}_iter_model_{test_idx}_{stress}_{angle}.png')

In [ ]:
_, bins, _ = plt.hist(np.array(losseses).std(axis=1), bins=10)
plt.clf()
logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
count, bins, _ = plt.hist(np.array(losseses).std(axis=1), bins=logbins, rwidth=0.5, log=False)
plt.xscale('log')
plt.title('Distribution of std(loss)')

In [ ]:
pd.options.display.float_format = '{:,.2e}'.format
pd.DataFrame([bins[:-1], bins[1:], count], index=['min', 'max', 'count']).T.astype({'count': 'int64'})

# Visualization

In [ ]:
model = torch.load(f'../ckpt_{config.name}/epoch_{config.test.epoch}.pt', map_location=torch.device(device))

In [ ]:
G, data = G_list[0], data_list[0]

In [ ]:
hidden = model(data, output_hidden=True, numpy=True)[1:10]
projected = list(map(pca_project, hidden))
for i in range(9):
    graph_vis(G, projected[i])

In [ ]:
torch.tensor([1,2,3]).square()

In [ ]:
config